# Introduction

In this section I will be using Folium to see if there is a relationship between the launch site and the landing success.

In [1]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

df = pd.read_csv('datasets/launch_data_falcon9_wiki.csv')

df.head()

,Flight_Number,Launch_Site,Payload,Payload_Mass,Orbit,Customer,Mission_Outcome,Booster_Version,Landing_Outcome,Latitude,Longitude,Datetime,Class
0,1,CCSFS SLC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,F9 v1.0 B0003.1,Failure (parachute),28.563197,-80.57682,2010-06-04 18:45:00,0
1,2,CCSFS SLC-40,Dragon,0.0,LEO,NASA (COTS) NRO,Success,F9 v1.0 B0004.1,Failure (parachute),28.563197,-80.57682,2010-12-08 15:43:00,0
2,3,CCSFS SLC-40,Dragon,525.0,LEO,NASA (COTS),Success,F9 v1.0 B0005.1,No attempt,28.563197,-80.57682,2012-05-22 07:44:00,0
3,4,CCSFS SLC-40,SpaceX CRS-1,4700.0,LEO,NASA (CRS),Success,F9 v1.0 B0006.1,No attempt,28.563197,-80.57682,2012-10-08 00:35:00,0
4,5,CCSFS SLC-40,SpaceX CRS-2,4877.0,LEO,NASA (CRS),Success,F9 v1.0 B0007.1,No attempt,28.563197,-80.57682,2013-03-01 15:10:00,0


## Task 1: Mark all launch sites on a map

In [2]:
# select relevant columns
spacex_df = df[['Launch_Site', 'Latitude', 'Longitude', 'Class']]

# create a dataframe of just the three launch sites and their locations
launch_sites_df = spacex_df.groupby('Launch_Site', as_index=False).first()
launch_sites_df = launch_sites_df[['Launch_Site', 'Latitude', 'Longitude']]

launch_sites_df

,Launch_Site,Latitude,Longitude
0,CCSFS LC-40,28.562302,-80.577356
1,CCSFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


First, let's create a Folium Map object with the NASA Johnson Space Center in Houston, Texas as the first pinpoint:

In [3]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

In [4]:
circle = folium.Circle(nasa_coordinate, radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(icon_size=(20,20),
                 icon_anchor=(0,0), 
                 html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC')
)

site_map.add_child(circle)
site_map.add_child(marker)

In [5]:
# add a circle and marker for each launch site
launch_map = folium.Map(location=nasa_coordinate, zoom_start=4)

for name, lat, long in zip(launch_sites_df['Launch_Site'], launch_sites_df['Latitude'], launch_sites_df['Longitude']):
    circle = folium.Circle([lat, long], radius=1000, 
                       color='#d35400', 
                       fill=True).add_child(folium.Popup(f'Launch from {name}'))

    marker = folium.map.Marker(
        [lat, long],
        icon=DivIcon(icon_size=(20,20),
                     icon_anchor=(0,0), 
                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name)
    )

    launch_map.add_child(circle)
    launch_map.add_child(marker)
    print(f'Added {name} to map.')

Added CCSFS LC-40 to map.
Added CCSFS SLC-40 to map.
Added KSC LC-39A to map.
Added VAFB SLC-4E to map.


In [6]:
launch_map

From the map, we can see that all the launch sites are close to the coasts, with VAFB being on the west coast of the United States and CCSFS and KSC both on the east. The launch sites are also fairly close to the equator, or at least in locations of the United States that are closest to the equator rather than the northern areas of the country.

## Task 2: Mark the success/failed launches for each site on the map

Markers for each launch record will be added to the map to see which sites have high success rates. A green marker will be used for a successful landing (Class = 1) and a red one for an unsuccessful landing (Class = 0).

Since a launch only happens in one of the three launch sites, there will be many launch records with the same coordinate. `MarkerClusters` will be useful here.

In [7]:
marker_cluster = MarkerCluster()

# create a column in the dataframe with the color for each launch
spacex_df['Color'] = spacex_df['Class'].map(lambda x: 'green' if x else 'red')

spacex_df.head()

/var/folders/0r/wbzfff2x7xn34__m82hhzyj40000gn/T/ipykernel_4508/2509329330.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spacex_df['Color'] = spacex_df['Class'].map(lambda x: 'green' if x else 'red')


,Launch_Site,Latitude,Longitude,Class,Color
0,CCSFS SLC-40,28.563197,-80.57682,0,red
1,CCSFS SLC-40,28.563197,-80.57682,0,red
2,CCSFS SLC-40,28.563197,-80.57682,0,red
3,CCSFS SLC-40,28.563197,-80.57682,0,red
4,CCSFS SLC-40,28.563197,-80.57682,0,red


In [8]:
for i, launch in spacex_df.iterrows():
    marker = folium.map.Marker(
        [launch.Latitude, launch.Longitude],
        icon=folium.Icon(color='white', icon_color=launch.Color)
    )
    marker_cluster.add_child(marker)
launch_map.add_child(marker_cluster)
launch_map

## Task 3: Calculate distances between a launch site and proximities

Being able to measure the distance between a launch site and nearby landmarks (such as the coastline or railroads) could prove to be valuable information. This functionality will be added to the Folium map using the `MousePosition` object.

In [9]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# add mouse position to get the coordinate for the cursor on the map
formatter = 'function(num) {return L.Util.formatNum(num, 5);};'  # javascript code
mouse_position = MousePosition(position='topright', 
                               separator=' Long: ', 
                               empty_string='NaN', 
                               lng_first=False, 
                               num_digits=20, 
                               prefix='Lat:', 
                               lat_formatter=formatter,
                               lng_formatter=formatter)

launch_map.add_child(mouse_position)
launch_map

Hovering the cursor over the map now shows the latitude and longitude of the cursor's location in the top right corner of the map.

We can zoom into the Kennedy Space Center and see how far away it is from some landmarks:

In [10]:
KSC = [28.573255, -80.646895]
CCSFS = [28.563197, -80.576820]
VAFB = [34.632834, -120.610745]

# distance from KSC to coastline
coast = [28.60517, -80.59126]
KSC_to_coast = calculate_distance(coast[0], coast[1], KSC[0], KSC[1])

# draw a marker to show distance
coast_mark = folium.Marker(coast, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(KSC_to_coast)))

# draw a Polyline between launch site and coastline point
line = folium.PolyLine(locations=[(coast[0], coast[1]), (KSC[0], KSC[1])], weight=1)

launch_map.add_child(coast_mark)
launch_map.add_child(line)

In [11]:
# distance from KSC to closest city
city = [28.61346, -80.80846]
KSC_to_city = calculate_distance(city[0], city[1], KSC[0], KSC[1])

# draw a marker to show distance
city_mark = folium.Marker(city, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(KSC_to_city)))

line = folium.PolyLine(locations=[(city[0], city[1]), (KSC[0], KSC[1])], weight=1)

launch_map.add_child(city_mark)
launch_map.add_child(line)

# distance from KSC to closest railyway
railway = [28.5732, -80.65417]
KSC_to_railway = calculate_distance(railway[0], railway[1], KSC[0], KSC[1])

# draw a marker to show distance
railway_mark = folium.Marker(railway, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(KSC_to_railway)))

line = folium.PolyLine(locations=[(railway[0], railway[1]), (KSC[0], KSC[1])], weight=1)

launch_map.add_child(railway_mark)
launch_map.add_child(line)

In [12]:
# distance from CCSFS to coastline
coast = [28.56278, -80.56787]
CCSFS_to_coast = calculate_distance(coast[0], coast[1], CCSFS[0], CCSFS[1])

# draw a marker to show distance
coast_mark = folium.Marker(coast, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(CCSFS_to_coast)))

# draw a Polyline between launch site and coastline point
line = folium.PolyLine(locations=[(coast[0], coast[1]), (CCSFS[0], CCSFS[1])], weight=1)

launch_map.add_child(coast_mark)
launch_map.add_child(line)

# distance from CCSFS to closest city
city = [28.40197, -80.60456]
CCSFS_to_city = calculate_distance(city[0], city[1], CCSFS[0], CCSFS[1])

# draw a marker to show distance
city_mark = folium.Marker(city, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(CCSFS_to_city)))

line = folium.PolyLine(locations=[(city[0], city[1]), (CCSFS[0], CCSFS[1])], weight=1)

launch_map.add_child(city_mark)
launch_map.add_child(line)

In [13]:
# distance from VAFB to coastline
coast = [34.63667, -120.62484]
VAFB_to_coast = calculate_distance(coast[0], coast[1], VAFB[0], VAFB[1])

# draw a marker to show distance
coast_mark = folium.Marker(coast, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(VAFB_to_coast)))

# draw a Polyline between launch site and coastline point
line = folium.PolyLine(locations=[(coast[0], coast[1]), (VAFB[0], VAFB[1])], weight=1)

launch_map.add_child(coast_mark)
launch_map.add_child(line)

# distance from VAFB to closest city
city = [34.63829, -120.45823]
VAFB_to_city = calculate_distance(city[0], city[1], VAFB[0], VAFB[1])

# draw a marker to show distance
city_mark = folium.Marker(city, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(VAFB_to_city)))

line = folium.PolyLine(locations=[(city[0], city[1]), (VAFB[0], VAFB[1])], weight=1)

launch_map.add_child(city_mark)
launch_map.add_child(line)

# distance from VAFB to closest railyway
railway = [34.63603, -120.62389]
VAFB_to_railway = calculate_distance(railway[0], railway[1], VAFB[0], VAFB[1])

# draw a marker to show distance
railway_mark = folium.Marker(railway, 
                           icon=DivIcon(icon_size=(20,20),
                                        icon_anchor=(0,0), 
                                        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(VAFB_to_railway)))

line = folium.PolyLine(locations=[(railway[0], railway[1]), (VAFB[0], VAFB[1])], weight=1)

launch_map.add_child(railway_mark)
launch_map.add_child(line)

From the distance markers we can see that the launch sites are generally in close proximity to railways, and coastlines and far away from cities and highways (at least 10km). This is likely a conscious decision meant to minimize damage from failed missions, noise, radio interference, etc. Having the launch sites close to the coast allows for rockets to be crashed into the ocean rather than somewhere populous.